In [89]:
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [90]:
fpath = "data/final_datasets/"
files = [fpath+f for f in os.listdir(fpath) if 'zscores' in f]
dfs = {file.split('_')[1].split('/')[-1]:pd.read_csv(file,sep=';') for file in files}

In [96]:
plt.style.use('fivethirtyeight')

bar_dict = {}
for ix,k in enumerate(dfs.keys()):
    
    
    vals = pd.DataFrame(dfs[k].corr()['Mean'].drop('Mean'))
    for key,val in vals.iterrows():
        if key not in bar_dict:
            bar_dict['period_type'] = []
            
            bar_dict[key] = [val['Mean']]
            
            
            
        else:
            bar_dict[key].append(val['Mean'])
            
    
    bar_dict['period_type'].append(k)

    
bar_df = pd.DataFrame(bar_dict).sort_values(['period_type']).set_index('period_type')
fig = px.bar(data_frame=bar_df, barmode="group")
fig.show()

In [92]:
dflist = []
for k,v in dfs.items():
    v['period_type'] = [k]*v.shape[0]
    dflist.append(v)
df = pd.concat(dflist).sort_values('date')


# Create figure with secondary y-axis
#fig = make_subplots(specs=[[{"secondary_y": True}]])

for dim in df.columns[1:-2]:
    subfig = make_subplots(specs=[[{"secondary_y": True}]])

    fig2 = px.line(df, x="date", y=dim)
    fig = px.line(df, x="date", y="Mean")
    fig2.update_traces(yaxis="y2")

    subfig.add_traces(fig.data + fig2.data)
    
    # recoloring is necessary otherwise lines from fig und fig2 would share each color
    # e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
    subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
    subfig.show()